Install Dependencies

In [51]:
pip install pandas rapidfuzz numpy

Note: you may need to restart the kernel to use updated packages.


Step 1: Load Data

In [52]:
import pandas as pd
import numpy as np
from rapidfuzz import fuzz
from rapidfuzz.distance import JaroWinkler


df = pd.read_csv("data/customers.csv")
df.fillna("", inplace=True)
df.head()

,customer_id,name,email,phone,address
0,1,Ramesh K N,ramesh@gmail.com,9876543210,Bangalore
1,2,Ramesh KN,ramesh@gmail.com,9876543210,Bengaluru
2,3,Ramesh Kumar,ramesh.k@gmail.com,9876543211,Bangalore
3,4,Suresh Kumar,suresh@gmail.com,9123456780,Mysore
4,5,Ramesh K.,ramesh@gmail.com,9876543210,Bengaluru


In [53]:
def normalize(text):
    return str(text).lower().strip()

for col in ["name", "email", "address"]:
    df[col] = df[col].apply(normalize)

Step 2: Blocking Strategy (Scalability)

We reduce comparisons by blocking on first letter of name + email domain.

In [54]:
def create_block_key(row):
    name_key = row['name'][:1].lower()
    email_key = row['email'].split('@')[-1] if '@' in row['email'] else ''
    return name_key + "_" + email_key

df["block_key"] = df.apply(create_block_key, axis=1)
df.head()


,customer_id,name,email,phone,address,block_key
0,1,ramesh k n,ramesh@gmail.com,9876543210,bangalore,r_gmail.com
1,2,ramesh kn,ramesh@gmail.com,9876543210,bengaluru,r_gmail.com
2,3,ramesh kumar,ramesh.k@gmail.com,9876543211,bangalore,r_gmail.com
3,4,suresh kumar,suresh@gmail.com,9123456780,mysore,s_gmail.com
4,5,ramesh k.,ramesh@gmail.com,9876543210,bengaluru,r_gmail.com


This reduces O(n²) comparisons drastically.

Step 3: Fuzzy Matching (RapidFuzz)

We combine multiple fields:

Name similarity

Email similarity

Phone exact match (strong signal)

In [ ]:
def compute_similarity(r1, r2):
    name_lev = fuzz.token_sort_ratio(r1['name'], r2['name'])
    name_jw = JaroWinkler.similarity(r1['name'], r2['name']) * 100
    email_score = fuzz.ratio(r1['email'], r2['email'])
    phone_score = 100 if r1['phone'] == r2['phone'] else 0

    final_score = (
        0.4 * name_lev +
        0.2 * name_jw +
        0.25 * email_score +
        0.15 * phone_score
    )
    return final_score



Step 4: Find Duplicate Pairs (Inside Blocks)

In [56]:
matches = []
THRESHOLD = 85

for _, block in df.groupby("block_key"):
    records = block.to_dict("records")
    for i in range(len(records)):
        for j in range(i+1, len(records)):
            score = compute_similarity(records[i], records[j])
            if score >= THRESHOLD:
                matches.append({
                    "id1": records[i]["customer_id"],
                    "id2": records[j]["customer_id"],
                    "score": score
                })

matches_df = pd.DataFrame(matches)
matches_df


,id1,id2,score
0,8,9,85.833333
1,16,17,89.269231
2,18,19,88.400000
3,14,15,87.519139
4,1,2,97.494737
5,1,5,92.439766
6,2,5,94.666667
7,3,6,96.351648
8,11,12,89.656566
9,4,7,90.666667


Conflict Resolution & Merge Logic
Rules Used (Explain in Interview)

✔ Prefer non-null values

✔ Prefer longer strings (more complete info)

✔ Keep earliest customer_id as master

In [57]:
def merge_records(group):
    merged = {}
    for col in group.columns:
        if col == "customer_id":
            merged[col] = group[col].min()
        else:
            values = group[col].astype(str).unique()
            merged[col] = max(values, key=len)
    return pd.Series(merged)


Apply Merging

In [58]:
duplicate_ids = set(matches_df['id2'])

unique_df = df[~df['customer_id'].isin(duplicate_ids)]


merged_df = (
    df[df["customer_id"].isin(duplicate_ids)]
    .groupby("block_key")
    .apply(merge_records)
    .reset_index(drop=True)   
)


final_df = pd.concat([unique_df, merged_df], ignore_index=True)
final_df["block_key"] = final_df.apply(create_block_key, axis=1)
final_df


,customer_id,name,email,phone,address,block_key
0,1,ramesh k n,ramesh@gmail.com,9876543210,bangalore,r_gmail.com
1,3,ramesh kumar,ramesh.k@gmail.com,9876543211,bangalore,r_gmail.com
2,4,suresh kumar,suresh@gmail.com,9123456780,mysore,s_gmail.com
3,8,anita sharma,anita@gmail.com,9988776655,delhi,a_gmail.com
4,10,anita sharma,anita.s@gmail.com,9988776656,delhi,a_gmail.com
5,11,rohit verma,rohit@gmail.com,8899776655,mumbai,r_gmail.com
6,13,rohit verma,rohit.v@gmail.com,8899776656,mumbai,r_gmail.com
7,14,pooja patel,pooja@gmail.com,7766554433,ahmedabad,p_gmail.com
8,16,amit singh,amit@gmail.com,6655443322,jaipur,a_gmail.com
9,18,neha joshi,neha@gmail.com,5544332211,pune,n_gmail.com


Evaluation Metrics (Must Include)

In [59]:
print("Total Records:", len(df))
print("Duplicate Pairs Found:", len(matches_df))
print("Final Unique Records:", len(final_df))


Total Records: 20
Duplicate Pairs Found: 10
Final Unique Records: 16


Shows impact of entity resolution clearly.

Output (Merged Clean Data)

In [60]:
final_df.to_csv("merged_customers.csv", index=False)


## Match Summary Report

- Total input records processed
- Duplicate pairs detected using fuzzy matching
- Final unique customer records after merge

### Matching Techniques
- Levenshtein distance (token-based similarity)
- Jaro-Winkler similarity for name comparison
- Blocking using name prefix and email domain

### Conflict Resolution Strategy
- Lowest customer_id retained as master record
- Non-null and longer values prioritized
- Deterministic merge logic

### Scalability
- Blocking significantly reduces pairwise comparisons
- Suitable for large customer datasets
